<a href="https://colab.research.google.com/github/raulbs7/Machine-Learning-Techniques-Project/blob/master/NLP_Supervised_Project/1_Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. NLP PREPROCESSING

## 1.1 Imports

In [1]:
!pip install tweet-preprocessor
!pip install emoji

     |████████████████████████████████| 51kB 4.6MB/s 
  Created wheel for emoji: filename=emoji-0.6.0-cp36-none-any.whl size=49717 sha256=f18802366723535194efcd9498317fd30d7fc4484f4f5d53b7ab6817c32b39ab
  Stored in directory: /root/.cache/pip/wheels/46/2c/8b/9dcf5216ca68e14e0320e283692dce8ae321cdc01e73e17796
Successfully built emoji


In [5]:
from google.colab import files
from google.colab import drive
import pandas as pd
import nltk
import io
import re
import string
import emoji
import functools
import operator

import preprocessor as p
from collections import Counter
from nltk.tokenize import sent_tokenize, word_tokenize

import html
import emo_unicode

from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

nltk.download("popular")
nltk.download('wordnet')
nltk.download('vader_lexicon') #sentiment analysis
nltk.download('twython') #twitter

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cmudict.zip.
[nltk_data]    | Downloading package gazetteers to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/gazetteers.zip.
[nltk_data]    | Downloading package genesis to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/genesis.zip.
[nltk_data]    | Downloading package gutenberg to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/gutenberg.zip.
[nltk_data]    | Downloading package inaugural to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/inaugural.zip.
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping corpora/movie_reviews.zip.
[nltk_data]    | Downloading package names to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/names.zip.
[nltk_data]    | Downloading package shakespeare to /root/nltk_data...
[nlt

False

## 1.2 Preprocessing steps

To preprocess the data, first it is needed to catch the data from the csv files.

In [6]:
def upload_dataframes (index_fields):
  uploaded = files.upload()
  for fn in uploaded.keys():
    print('User uploaded file "{name}" with length {length} bytes'.format(
        name=fn, length=len(uploaded[fn])))
    df = pd.read_csv(io.StringIO(uploaded[fn].decode('utf-8')), index_col = index_fields)
    return df

The csv file that is going to be treated to the preprocessing is **labeled_data.csv**

In [7]:
tweets = upload_dataframes([])

Saving labeled_data.csv to labeled_data.csv
User uploaded file "labeled_data.csv" with length 2546446 bytes


In [8]:
print(tweets.shape)
tweets.head()

(24783, 7)


,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet
0,0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...


### 1.2.1 Removing mentions
For this data, it is important to remove some concepts like mentions that they doesn't give us any additional information and which makes much noise, or it is difficult to treat.

In [9]:
def remove_mentions(text):
  p.set_options(p.OPT.MENTION)
  return p.clean(text)

In [10]:
tweets['tweet'] = tweets['tweet'].apply(remove_mentions)
tweets.head()

,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet
0,0,3,0,0,3,2,!!! RT : As a woman you shouldn't complain abo...
1,1,3,0,3,0,1,!!!!! RT : boy dats cold...tyga dwn bad for cu...
2,2,3,0,3,0,1,!!!!!!! RT Dawg!!!! RT : You ever fuck a bitch...
3,3,3,0,2,1,1,!!!!!!!!! RT : she look like a tranny
4,4,6,0,6,0,1,!!!!!!!!!!!!! RT : The shit you hear about me ...


### 1.2.2 Removing URLs
There are some tweets that have URLs that can be removed in order to not have conflicts later.

In [11]:
def remove_urls(text):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

In [12]:
tweets['tweet'] = tweets['tweet'].apply(remove_urls)
tweets.head()

,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet
0,0,3,0,0,3,2,!!! RT : As a woman you shouldn't complain abo...
1,1,3,0,3,0,1,!!!!! RT : boy dats cold...tyga dwn bad for cu...
2,2,3,0,3,0,1,!!!!!!! RT Dawg!!!! RT : You ever fuck a bitch...
3,3,3,0,2,1,1,!!!!!!!!! RT : she look like a tranny
4,4,6,0,6,0,1,!!!!!!!!!!!!! RT : The shit you hear about me ...


### 1.2.3 Removal of numbers

In this project we have considered numbers as noise that could provoke more number of features in the next steps, so they are going to be removed.

In [16]:
def remove_numbers(text):
  return re.sub('\d+', '', text)

In [17]:
tweets['tweet'] = tweets['tweet'].apply(remove_numbers)
tweets.head()

,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet
0,0,3,0,0,3,2,!!! RT : As a woman you shouldn't complain abo...
1,1,3,0,3,0,1,!!!!! RT : boy dats cold...tyga dwn bad for cu...
2,2,3,0,3,0,1,!!!!!!! RT Dawg!!!! RT : You ever fuck a bitch...
3,3,3,0,2,1,1,!!!!!!!!! RT : she look like a tranny
4,4,6,0,6,0,1,!!!!!!!!!!!!! RT : The shit you hear about me ...


### 1.2.4 Removal of capital letters

Next step in preprocessing is to remove the capital letters.

In [18]:
def remove_capital_let(text):
  return text.lower()

In [19]:
tweets['tweet'] = tweets['tweet'].apply(remove_capital_let)
tweets.head()

,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet
0,0,3,0,0,3,2,!!! rt : as a woman you shouldn't complain abo...
1,1,3,0,3,0,1,!!!!! rt : boy dats cold...tyga dwn bad for cu...
2,2,3,0,3,0,1,!!!!!!! rt dawg!!!! rt : you ever fuck a bitch...
3,3,3,0,2,1,1,!!!!!!!!! rt : she look like a tranny
4,4,6,0,6,0,1,!!!!!!!!!!!!! rt : the shit you hear about me ...


### 1.2.5 Removal of contractions

Next, it can be optional to remove contractions to have a better tokenization.

In [20]:
def remove_contractions(text):
  text = html.unescape(text) #Converting apostrophes
  text = re.sub(r'’', '\'', text)

  text = re.sub(r'isn\'t', r'is not', text)
  text = re.sub(r'aren\'t', r'are not', text)
  text = re.sub(r'wasn\'t', r'was not', text)
  text = re.sub(r'hadn\'t', r'had not', text)
  text = re.sub(r'haven\'t', r'have not', text)
  text = re.sub(r'wasn\'t', r'was not', text)
  text = re.sub(r'weren\'t', r'were not', text)
  text = re.sub(r'can\'t', r'can not', text)
  text = re.sub(r'doesn\'t', r'does not', text)
  text = re.sub(r'couldn\'t', r'could not', text)
  text = re.sub(r'shouldn\'t', r'should not', text)
  text = re.sub(r'don\'t', r'do not', text)
  text = re.sub(r'didn\'t', r'did not', text)
  text = re.sub(r'mustn\'t', r'must not', text)
  text = re.sub(r'might\'ve', r'might not', text)
  text = re.sub(r'won\'t', r'will not', text)
  text = re.sub(r'wouldn\'t', r'would not', text)
  text = re.sub(r'what\'s', r'what is', text)
  text = re.sub(r'where\'s', r'where is', text)
  text = re.sub(r'who\'s', r'who is', text)
  text = re.sub(r'who\'ll', r'who will', text)
  text = re.sub(r'that\'s', r'that is', text)
  text = re.sub(r'there\'s', r'there is', text)
  text = re.sub(r'i\'ll', r'i will', text)
  text = re.sub(r'i\'m', r'i am', text)
  text = re.sub(r'i\'have', r'i have', text)
  text = re.sub(r'i\'d', r'i would', text)
  #text = re.sub(r'i\'d', r'i had', text)
  text = re.sub(r'you\'re', r'you are', text)
  text = re.sub(r'you\'ve', r'you have', text)
  text = re.sub(r'you\'ll', r'you will', text)
  text = re.sub(r'you\'d', r'you would', text)
  #text = re.sub(r'you\'d', r'you had', text)
  text = re.sub(r'he\'s', r'he is', text)
  text = re.sub(r'he\'ll', r'he will', text)
  text = re.sub(r'she\'s', r'she is', text)
  text = re.sub(r'she\'ll', r'she will', text)
  text = re.sub(r'it\'s', r'it is', text)
  text = re.sub(r'it\'ll', r'it will', text)
  text = re.sub(r'we\'re', r'we are', text)
  text = re.sub(r'we\'ve', r'we have', text)
  text = re.sub(r'we\'ll', r'we will', text)
  text = re.sub(r'we\'d', r'we would', text)
  #text = re.sub(r'we\'d', r'we had', text)
  text = re.sub(r'they\'re', r'they are', text)
  text = re.sub(r'they\'ve', r'they have', text)
  text = re.sub(r'they\'ll', r'they will', text)
  text = re.sub(r'they\'d', r'they would', text)
  #text = re.sub(r'they\'d', r'they had', text)

  return text

In [21]:
tweets['tweet'] = tweets['tweet'].apply(remove_contractions)
tweets.head()

,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet
0,0,3,0,0,3,2,!!! rt : as a woman you should not complain ab...
1,1,3,0,3,0,1,!!!!! rt : boy dats cold...tyga dwn bad for cu...
2,2,3,0,3,0,1,!!!!!!! rt dawg!!!! rt : you ever fuck a bitch...
3,3,3,0,2,1,1,!!!!!!!!! rt : she look like a tranny
4,4,6,0,6,0,1,!!!!!!!!!!!!! rt : the shit you hear about me ...


### 1.2.6 Removing symbols

One important thing is to remove symbols which are noise and not provide any information to the analysis of the text. The symbols that they are going to be removed are the next ones: !"#$%&()*+,-./:;<=>?@[\\]^_{|}~`’«»

It is important to converse in UNICODE form the possible emojis, that can appear in some tweets, to preserve them from the removal of those symbols.

In [22]:
SYMBOLS_TO_REMOVE = re.escape('!"#$%&()*+,-./:;<=>?@[\\]^_{|}~`\'“”’«»')

def remove_symbols(text):
  text = html.unescape(text) #Converting emojis
  return re.sub(f'[{SYMBOLS_TO_REMOVE}]', '', text)

In [23]:
tweets['tweet'] = tweets['tweet'].apply(remove_symbols)
tweets.head()

,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet
0,0,3,0,0,3,2,rt as a woman you should not complain about ...
1,1,3,0,3,0,1,rt boy dats coldtyga dwn bad for cuffin dat ...
2,2,3,0,3,0,1,rt dawg rt you ever fuck a bitch and she sta...
3,3,3,0,2,1,1,rt she look like a tranny
4,4,6,0,6,0,1,rt the shit you hear about me might be true ...


### 1.2.7 Removal of repeated words

In [24]:
def remove_repeated(text):
  return re.sub(r'\b(\w+)( \1\b)+', r'\1', text)

There is an example to see how the previous function works because its complicated to show the effect in the dataset of tweets.

In [25]:
print(remove_repeated("hello hello 1 1 how how are are you you ?"))

tweets['tweet'] = tweets['tweet'].apply(remove_repeated)
tweets.head()

hello 1 how are you ?


,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet
0,0,3,0,0,3,2,rt as a woman you should not complain about ...
1,1,3,0,3,0,1,rt boy dats coldtyga dwn bad for cuffin dat ...
2,2,3,0,3,0,1,rt dawg rt you ever fuck a bitch and she sta...
3,3,3,0,2,1,1,rt she look like a tranny
4,4,6,0,6,0,1,rt the shit you hear about me might be true ...


### 1.2.8 Tokenization

In order to determinine the lemma of a word based on its intended meaning, the following step must be carried out:

In [26]:
# 1. Tokenization

def tokenization_words(raw):
  tokens = word_tokenize(raw)
  tokens
  return tokens

def tokenization_sent(raw):
  tokens = sent_tokenize(raw)
  tokens
  return tokens

In [27]:
tweets_words = tweets['tweet'].apply(tokenization_words)

tweets_words.head()

0    [rt, as, a, woman, you, should, not, complain,...
1    [rt, boy, dats, coldtyga, dwn, bad, for, cuffi...
2    [rt, dawg, rt, you, ever, fuck, a, bitch, and,...
3                     [rt, she, look, like, a, tranny]
4    [rt, the, shit, you, hear, about, me, might, b...
Name: tweet, dtype: object

In [28]:
tweets_sent = tweets['tweet'].apply(tokenization_sent)
tweets_sent.head()

0    [ rt  as a woman you should not complain about...
1    [ rt  boy dats coldtyga dwn bad for cuffin dat...
2    [ rt dawg rt  you ever fuck a bitch and she st...
3                        [ rt  she look like a tranny]
4    [ rt  the shit you hear about me might be true...
Name: tweet, dtype: object

### 1.2.9 Stopwords

The stop words have to be filtered out before lemmatization. Stop words refers to the most common words in a language as prepositions or determiners.

In [29]:
def stop_words(text):
  #STOPWORDS
  english_stops = set(stopwords.words('english'))
  clean_sent = [word for word in text if word not in english_stops]
  return clean_sent
  #STOPWORDS 

In [30]:
tweets_clean = tweets_words.apply(stop_words)
tweets_clean.head()

0    [rt, woman, complain, cleaning, house, man, al...
1    [rt, boy, dats, coldtyga, dwn, bad, cuffin, da...
2    [rt, dawg, rt, ever, fuck, bitch, start, cry, ...
3                             [rt, look, like, tranny]
4    [rt, shit, hear, might, true, might, faker, bi...
Name: tweet, dtype: object

### 1.2.10 Conversion of emojis

Due to the fact that tweets are being treated, there is a lot of probability to have some emojis in our data. These emojis are represented in our data as a code. Our goal, is to convert this emojis in tokenizable and meaningful words.

There is a criteria when it appears a bunch of emojis next each other, that would be choosing the first emoji of this set. This is a criteria made due to the difficult ot treatment of having many emojis. For this reason, the decision is to take the first because it is normally the most relevant, and beacuse same emojis are repeated a lot of times in messages.

In [31]:
def convert_emojis(text):
  
  em_split_emoji = emoji.get_emoji_regexp().split(text)
  em_split_whitespace = [substr.split() for substr in em_split_emoji]
  em_split = functools.reduce(operator.concat, em_split_whitespace)

  text = html.unescape(em_split[0])
  
  if len(em_split) > 0:
    if em_split[0] in emo_unicode.UNICODE_EMO:
      text = re.sub(re.escape('{}'.format(em_split[0])), r'_'.join(emo_unicode.UNICODE_EMO[em_split[0]].replace(r",",r"").replace(r":",r"").split()), em_split[0])
    else:
      pattern_others = re.compile(pattern = "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags = re.UNICODE)
      text = pattern_others.sub(r'',text)
  else:
    text = ''
  return text

def treat_emojis(array):
  for i in range(len(array)):
    array[i] = convert_emojis(array[i])
  return array

In [32]:
tweets_clean = tweets_clean.apply(treat_emojis)
tweets_clean.head()

0    [rt, woman, complain, cleaning, house, man, al...
1    [rt, boy, dats, coldtyga, dwn, bad, cuffin, da...
2    [rt, dawg, rt, ever, fuck, bitch, start, cry, ...
3                             [rt, look, like, tranny]
4    [rt, shit, hear, might, true, might, faker, bi...
Name: tweet, dtype: object

### 1.2.11 Correction of wrong words

After that, it is necessary to correct wrong words of the text, in order to have a better analysis and to create a text sequence more understandable for our NLP algorithms. The file used to extract correct words is **big.txt**

In [33]:
uploaded = files.upload()
file_name = "big.txt"
raw = uploaded[file_name].decode("utf-8")

Saving big.txt to big.txt


In [34]:
def words(text): return re.findall(r'\w+', text.lower())

WORDS = Counter(words(raw))

def P(word, N=sum(WORDS.values())): 
    "Probability of `word`."
    return WORDS[word] / N

def correction(word): 
    "Most probable spelling correction for word."
    return max(candidates(word), key=P)

def candidates(word): 
    "Generate possible spelling corrections for word."
    return (known([word]) or known(edits1(word)) or known(edits2(word)) or [word])

def known(words): 
    "The subset of `words` that appear in the dictionary of WORDS."
    return set(w for w in words if w in WORDS)

def edits1(word):
    "All edits that are one edit away from `word`."
    letters    = 'abcdefghijklmnopqrstuvwxyz'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

def edits2(word): 
    "All edits that are two edits away from `word`."
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))

In [35]:
def treat_correction(array):
  for i in range(len(array)):
    array[i] = correction(array[i])
  return array

The correction of tweets is not executed because it converts rt's and other important words in other words.

In [36]:
#tweets_clean = tweets_clean.apply(treat_correction)
#tweets_clean.head()

### 1.2.12 Lemmatization

Lemmatization consists into obtain the root of the words, so it will cause having less variety of vocabulary.

In [37]:
def lemmatizer_words(text):
  lemmatizer = WordNetLemmatizer()
  return [lemmatizer.lemmatize(word) for word in text]

In [38]:
tweets_lemmatizer=tweets_clean.apply(lemmatizer_words)
tweets_lemmatizer.head()

0    [rt, woman, complain, cleaning, house, man, al...
1    [rt, boy, dat, coldtyga, dwn, bad, cuffin, dat...
2    [rt, dawg, rt, ever, fuck, bitch, start, cry, ...
3                             [rt, look, like, tranny]
4    [rt, shit, hear, might, true, might, faker, bi...
Name: tweet, dtype: object

### 1.2.13 Prepare dataframe

In [39]:
def list_to_string(list):
  return ' '.join(list)

In [40]:
tweets['tweet'] = tweets_lemmatizer.apply(list_to_string)
tweets.head()

,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet
0,0,3,0,0,3,2,rt woman complain cleaning house man always ta...
1,1,3,0,3,0,1,rt boy dat coldtyga dwn bad cuffin dat hoe st ...
2,2,3,0,3,0,1,rt dawg rt ever fuck bitch start cry confused ...
3,3,3,0,2,1,1,rt look like tranny
4,4,6,0,6,0,1,rt shit hear might true might faker bitch told ya


In [41]:
tweets = tweets.drop('Unnamed: 0', axis=1)

To conclude this phase, it is necessary to export the dataframe containing the preprocessed tweets to treat them.

In [42]:
drive.mount('/drive')
tweets.to_csv('/drive/My Drive/processed_tweets.csv')

Mounted at /drive
